# Steps to reproduce this environment to be able to use openmm-torch
Disclaimer: All solutions and fixes were suggestions found in github forums adapted for the present problem.

1. Install pytorch 1.13.0 (preferrable from conda-forge channel), using CUDA 11.7 version.
2. Install openmm from conda as well
3. Try to install openmm-torch with conda
4. Fail and die
5. Rise from the ashes like a phoenix and install openmm-torch from source like a pro (steps to install below) (to confirm if we need to install from source)
6. Install caffee from conda
7. Add the following env variables: `export LD_LIBRARY_PATH=/path/to/caffe2/build/lib:$LD_LIBRARY_PATH` and `export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64:$LD_LIBRARY_PATH`
8. By this point you will have to recompile openmm-torch
9. In my case (Maybe not needed at all) had to uninstall pytorch and install it again.

Note: You need to import torch before importing openmmtorch, otherwise some features won't work

### Steps to install openmm-torch from source

1. Go to `https://pytorch.org/get-started/locally/` and select the combination: Stable, Linux, Libtorch, C++/Java, CUDA 11.7, and copy the command below the description "Download here (cxx11 ABI):". Then change the version from 2.0.1 to 1.13.0. Or alternatively, just copy the following command in the terminal: `wget https://download.pytorch.org/libtorch/cu117/libtorch-shared-with-deps-1.13.0%2Bcu117.zip`
2. unzip the zipped file
3. Make sure the folder where you unzipped libtorch (The thing downloaded above) is accessible from where you're going to build openmm-torch
4. Clone locally the repo https://github.com/openmm/openmm-torch
5. write in the terminal `ccmake openmm-torch`
6. In the interactive terminal, set `PYTORCH_DIR` to point to the directory where you installed the LibTorch.
7. Set `OPENMM_DIR` to point to the directory where OpenMM is installed. This is needed to locate the OpenMM header files and libraries. If you are unsure of what directory this is, the following script will print it out. Set the `CMAKE_INSTALL_PREFIX` to point to the same directory as the `OPENMM_DIR`.
8. Set the `CUDA_TOOLKIT_ROOT_DIR` because in this case we need it. And make sure `NN_BUILD_CUDA_LIB` is selected. In our case we also selected the flags `NN_BUILD_OPENCL_LIB`, `NN_BUILD_PYTHON_WRAPPERS` and the `CUDNN_STATIC`.
9. If it's the first time configuring the build, you may need to press the "configure" `[c]` option twice. After that the "generate" `[g]` option will appear. If the progress percentage reaches 100%


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np

from pathlib import Path
from openmm.app import *
from openmm import *
from openmm.unit import kelvin, picoseconds

from simutils import ForceReporter, build_system

In [ ]:
args_dict = {
    'mapping': 'martini3',
    'input': 'chignolin.data.test.pdb',
    'selection': 'all',
    'isatomistic': False,        # If the input pdb is an atomistic pdb, set this to True. It will save a CG version and load it in OpenMM
    'pos2unit': AngstromsPerNm,  # If the model works in Angstrom use this, if it works in nm just comment this line
    'energy2unit': KJPerKcal,    # If the model works in Kcal use this, if it works in KJ just comment this line
    'model': 'models/A2A.A.kcal.best.pt', # Path to the compiled model saved after training
    'sim_folder': 'simulations', # Save simulation and logs inside this folder
    'log_every': 100,            # Log every N simulation steps
    'steps': 10000,              # Simulation steps
}

In [ ]:
system, pdb = build_system(args_dict)

In [ ]:
# integrator = LangevinIntegrator(310*kelvin, 1./picoseconds , 0.01*picoseconds)
# integrator = VerletIntegrator(0.01*picoseconds)
integrator = NoseHooverIntegrator(310*kelvin, 10./picoseconds, 0.005*picoseconds)
platform = Platform.getPlatformByName('CPU')

simulation = Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.getPositions())

# Create simulaiton folder - #

p = Path(args_dict.get('model'))
sim_root = os.path.join(args_dict.get('sim_folder', 'simulations'), str(p.stem))
os.makedirs(str(sim_root), exist_ok=True)

# - Save initial structure to a PDB file inside simulation folder - #

PDBFile.writeFile(pdb.topology, pdb.getPositions(), open(os.path.join(sim_root, 'initial.pdb'), 'w'))

# Performs a local energy minimization.                                  #
# It is usually a good idea to do this at the start of a simulation,     #
# since the coordinates in the PDB file might produce very large forces. #

print("Minimizing energy...")
simulation.minimizeEnergy()

# - Write minimized system to a PDB file - #
minimized_positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(pdb.topology, minimized_positions, open(os.path.join(sim_root, 'minimized.pdb'), 'w'))

# - Logging during simulation - #

log_every = args_dict.get('log_every', 1000)
simulation.reporters.append(PDBReporter(
    os.path.join(sim_root, 'output.pdb'), log_every
))
simulation.reporters.append(StateDataReporter(
    os.path.join(sim_root, 'output.dat'), log_every, step=True, potentialEnergy=True,
    kineticEnergy=True, temperature=True, time=True, totalEnergy=True
))
simulation.reporters.append(ForceReporter(
    os.path.join(sim_root, 'outputforces.txt'), log_every
))

# - Start simulation - #

print("Starting simulation...")
simulation.step(args_dict.get('steps'))
print("Simulation ended!")

# state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
# np.array([[a.x,a.y,a.z]for a in state.getForces()])[:5]

In [ ]:
# MOVE TO ANALYSIS NOTEBOOK #

# output_file = os.path.join(dataset_dir, "enere@usi.ch/CGffap/dataset.notVoid.npz")  #fmartip/ff-naive/DatasetsA2A/dataset.A2a.hydrogen.pose0.npz
# dataset = dict(np.load(output_file))

# np.linalg.norm(dataset['bead_forces'],axis=0).max()

# dataset_void = dict(np.load('/home/enere@usi.ch/CGffap/dataset.VoidNoPBC.npz'))
# np.linalg.norm(dataset_void['bead_forces'],axis=0).max()

# simForces = np.linalg.norm(np.array(simulation.reporters[2].getForces()),axis=-1)
# beadForces = np.linalg.norm(dataset['bead_forces'][:len(simForces)],axis=-1)
# beadForces_void = np.linalg.norm(dataset_void['bead_forces'][:len(simForces)],axis=-1)

# # Plot losses
# import matplotlib.pyplot as plt
# selected_bead_index = 0
# # Plotting dataset 1
# plt.figure(figsize=(10, 5))
# plt.plot(simForces[:,selected_bead_index], label='simForces')


# # Plotting dataset 2
# plt.plot(beadForces[:,selected_bead_index], linestyle='--', label='beadForces')

# # Plotting dataset 3
# # plt.plot(beadForces_void[:,selected_bead_index], label='beadForces void')

# # Adding labels and title
# plt.xlabel('Index')
# plt.ylabel('Force')
# plt.title('Comparison of Forces for Dataset 1 and Dataset 2')

# # Adding legend
# plt.legend()

# # Displaying the plot
# plt.show()